In [1]:
import numpy as np
import pytest
from qtm.circuits.kernel import AngleEmbeddingKernel

In [2]:

kernel = AngleEmbeddingKernel(
        feature_dimension=3,
        num_qubits=5,
        reps=2,
        entangle_inputs=[],
        noise_type= None,
        noise_params= None,
        reupload=False
    )

In [3]:
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np

# Generate data
X, y = make_classification(
    n_samples=100,
    n_features=2,          # IMPORTANT: match feature_dimension=3
    n_informative=2,
    n_redundant=0,
    n_clusters_per_class=2,
    n_classes=2,
    class_sep=0.5,
    random_state=42
)

y = np.where(y == 0, -1, 1)

In [4]:
kernel.parameter_shape()  # Should match the weights shape defined earlier

{'reps': 2,
 'rotation_weights_shape': (5,),
 'entangling_weights_shape': (4,),
 'full_structure': [((5,), (4,)), ((5,), (4,))]}

In [5]:
from qtm.models.qkernel import QKernel

model = QKernel(
                kernel_circuit = kernel,
                matrix_type = "regular",
                matrix_normalisation = False,
                landmark_points = 0,
                parallelise = True,
)

In [6]:
param_shape = kernel.parameter_shape()

reps = param_shape["reps"]
rot_shape = param_shape["rotation_weights_shape"]
ent_shape = param_shape["entangling_weights_shape"]

_weights = []

for _ in range(reps):
    w_rot = np.random.randn(*rot_shape)
    w_ent = np.random.randn(*ent_shape)
    _weights.append((w_rot, w_ent))

#model.execute(X, X, _weights)  # Compute kernel matrix for first 5 samples

In [ ]:
from qtm.trainer.ccka import CentroidKernelTrainer

trainer = CentroidKernelTrainer(model = model,
                                optimizer = "spsa",
                                outer_steps = 5,
                                inner_steps = 5,
                                num_subcentroids = 2,
                                lr_param = 0.3,
                                lr_centroid = 0.5,
                                lr_subcentroid = 0.5,
                                lambda_kao = 0.1,
                                lambda_co = 0.1,
                                X = X,
                                y = y,
                                training_split = 0.8,
                                validation_split = 0.2
                            )

In [8]:
metrics = trainer.evaluate()
print(metrics)

Kernel alignment (train):  0.2083346509662023
Condition number (train):  27064426776103.78
Accuracy (test):  0.9
Precision (test):  0.898989898989899
Recall (test):  0.898989898989899
F1-score (test):  0.898989898989899
SVM Margin:  0.04874441934714419
Number of support vectors:  33
{'kernel_alignment': 0.2083346509662023, 'min_eigenvalue': -0.1136930554810185, 'num_negative_eigenvalues': 25, 'psd_violation_magnitude': 1.2406048709939332, 'condition_number': 27064426776103.78, 'rank': 39, 'effective_rank': 6.695317387281786, 'accuracy': 0.9, 'precision': 0.898989898989899, 'recall': 0.898989898989899, 'f1': 0.898989898989899, 'svm_margin': 0.04874441934714419, 'num_support_vectors': 33}


In [ ]:
trainer.train()

-----------------------------------------------------------
=== Outer Step 1 / 5 ===
Class -1: Parameter optimization over 4 subcentroids
Class 1: Centroid optimization over 4 subcentroids
-----------------------------------------------------------
Class 1: Parameter optimization over 4 subcentroids
Class -1: Centroid optimization over 4 subcentroids
-----------------------------------------------------------
┌─────────────────────┐
│ KTA       : 0.2088  │
│ Accuracy  : 0.8750  │
│ Precision : 0.9091  │
│ Recall    : 0.8571  │
│ F1-score  : 0.8667  │
└─────────────────────┘
-----------------------------------------------------------
=== Outer Step 2 / 5 ===
Class -1: Parameter optimization over 4 subcentroids
Class 1: Centroid optimization over 4 subcentroids


In [ ]:
metrics = trainer.evaluate()
print(metrics)